Importing necessary packages. We use pandas for generating a dataframe and modifying it as required. The requests package is used to get a response object. Lh is used to extract table data based on html header tr.

In [0]:
import pandas as pd
import requests
import lxml.html as lh

Url is the website from which we want to scrape data, in our case the wikipedia site with a table file with postal codes, boroughs and neighbourhoods. the xpath parameter is to indicate the content we want to extract. tr implies the rows of the table

In [0]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Here we iterate over the elements at the 0th position since it contains the different columns of the table

In [5]:
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    name=name[:len(name)-1:]
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


Now we iterate over the rest of the tr elements. the length shouldn't be greater than the number of columns. We check if it's empty and do numerical and string conversions

In [0]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        st=str(data)
        st=st[:len(st)-1:]
        col[i][1].append(st)
        i+=1

We convert the the title and column lists to a dictionary where the column name is key and lists extracted above are the value. We convert this to a dataframe and drop all the rows where the borough isn't assigned and rename Postal Code to remove the space

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df_filtered=df[df['Borough']!="Not assigned"]
df_filtered.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_filtered.head(12)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
df_filtered.shape

(104, 3)